In [1]:
import os
import whisper
from jiwer import wer

In [ ]:
audio_dir = "outputs/test_dataset_v1/speech"
text_dir = "outputs/test_dataset_v1/text"

audio_files = sorted(f for f in os.listdir(audio_dir) if f.startswith('speech_') and f.endswith('.wav'))

all_hypotheses = []
all_references = []

In [ ]:
model = whisper.load_model("large")

for audio_file in audio_files:
    idx = audio_file.split('_')[1].split('.')[0]  
    audio_path = os.path.join(audio_dir, audio_file)
    text_file = f"text_{idx}.txt"
    text_path = os.path.join(text_dir, text_file)

   
    if not os.path.exists(text_path):
        print(f"Warning: Reference {text_file} not found for {audio_file}. Skipping.")
        continue

    with open(text_path, 'r', encoding='utf-8') as f:
        reference = f.read().strip()

    # Transcribe audio
    result = model.transcribe(audio_path)
    hypothesis = result["text"].strip()

    all_references.append(reference)
    all_hypotheses.append(hypothesis)

    print(f"Audio: {audio_file}")
    print(f"GT:   {reference}")
    print(f"Hyp:  {hypothesis}")
    print("-"*20)

100%|█████████████████████████████████████| 2.88G/2.88G [01:35<00:00, 32.3MiB/s]
/home/mkudriashov/tts/venv/lib/python3.10/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues relat

Audio: speech_0.wav
GT:   Коллеги, начнем с анализа объемов добычи, переработки и транспортировки углеводородов за прошедший квартал
Hyp:  Коллеги, начнем с анализа объемов добычи, переработки и транспортировки углеводородов за прошедший квартал.
--------------------
Audio: speech_1.wav
GT:   Согласно отчету производственного департамента, общий объем добычи нефти составил 12,8 млн тонн, что на 3% превышает плановый показатель
Hyp:  Согласно отчету производственного департамента, общий объем добычи нефти составил 12,8 млн тонн, что на 3% превышает плановый показатель.
--------------------
Audio: speech_10.wav
GT:   Это может повлиять на транспортную нагрузку и безопасность эксплуатации
Hyp:  Это может повлиять на транспортную нагрузку и безопасность эксплуатации.
--------------------
Audio: speech_100.wav
GT:   Также важными элементами системы управления рисками должны стать адаптация корпоративной структуры и процессов к новым рыночным условиям, что позволит свести к минимуму негативн

In [ ]:
import os
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import librosa
from jiwer import wer

audio_dir = "outputs/speech"
text_dir = "outputs/text"
model_dir = "./whisper-finetuned"  # or your saved directory

# Load fine-tuned model and processor
processor = WhisperProcessor.from_pretrained(model_dir)
model = WhisperForConditionalGeneration.from_pretrained(model_dir).to("cuda" if torch.cuda.is_available() else "cpu")

all_hypotheses = []
all_references = []

for audio_file in sorted(f for f in os.listdir(audio_dir) if f.startswith('speech_') and f.endswith('.wav')):
    idx = audio_file.split('_')[1].split('.')[0]
    text_path = os.path.join(text_dir, f"text_{idx}.txt")
    if not os.path.exists(text_path):
        continue
    with open(text_path, encoding="utf-8") as f:
        reference = f.read().strip()

    audio_path = os.path.join(audio_dir, audio_file)
    audio, sr = librosa.load(audio_path, sr=16000)
    input_features = processor(audio, sampling_rate=16000, return_tensors="pt").input_features.to(model.device)
    
    # Generate predicted text
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0].strip()
    
    all_hypotheses.append(transcription)
    all_references.append(reference)

    print(f"\nAudio: {audio_file}")
    print(f"GT:   {reference}")
    print(f"Hyp:  {transcription}")

# Calculate WER
score = wer(all_references, all_hypotheses)
print(f"\nCorpus Word Error Rate (WER): {score * 100:.2f}%")



Corpus Word Error Rate (WER): 9.50%
